# Predictor Local

Spin up container in the code root directory"
```bash 
docker-compose -f docker-compose.serve.yaml up
```

Run the code below to carry out inference:

In [ ]:
import pandas as pd
import numpy as np
from sklearn import metrics
from sagemaker import serializers
import requests
import yaml

In [ ]:
params = yaml.safe_load(open('../code/params.yaml'))
hyperparameters = params["hyperparameters"]
directories = params["directories"]
datasets = params["datasets"]
sagemaker_params = yaml.safe_load(open('../params.yaml'))
train_params = sagemaker_params["train"]
model_params = sagemaker_params["model"]
inference_params = sagemaker_params["inference"]

In [16]:
df = pd.read_csv("../data/test/test.csv")
df = df.drop(datasets['drop_columns'], axis=1)

In [17]:
X_test = df.drop(datasets['y_column'], axis=1)
y_true = df[datasets['y_column']]

In [ ]:
url = 'http://localhost:8080/invocations'
headers = {'Content-type': "application/x-npy"}
serializer=serializers.NumpySerializer()
response = requests.post(url, headers=headers, data=serializer.serialize(X_test))

In [ ]:
y_pred = response.json()

In [ ]:
mae = metrics.mean_absolute_error(y_true, y_pred)
rmse = (np.sqrt(metrics.mean_squared_error(y_true, y_pred)))
r2 = metrics.r2_score(y_true, y_pred)

print("Testing performance")
print(f"MAE: {mae:.2f}")
print(f"RMSE: {rmse:.2f}")
print(f"R2: {r2:.2f}")